### Naver Rest API
- 파파고 번역 api
- 통합검색어 트렌드 api

In [1]:
import warnings
warnings.filterwarnings('ignore')
import requests
import pandas as pd

#### 0. request token 얻기
- `https://developers.naver.com`
- 1. Request Token 얻기 : 애플리케이션등록 -> app_key 획득
- 2. app_key를 이용해서 데이터 가져오기

In [5]:
client_id = '내꺼'
client_secret = '내꺼'

#### 1. 파파고 번역 api
- `https://developers.naver.com/docs/papago/`
- 사용법
    - `https://developers.naver.com/docs/papago/papago-nmt-api-reference.md`

In [2]:
# 1. Document > URL
url = 'https://openapi.naver.com/v1/papago/n2mt'

In [3]:
# 2. Request(URL) > Response(JSON(str))
ko_txt = '데이터 크롤링은 돈이 됩니다.'
params = {
    'source' : 'ko',
    'target' : 'en',
    'text' : ko_txt,
}
params

{'source': 'ko', 'target': 'en', 'text': '데이터 크롤링은 돈이 됩니다.'}

In [6]:
headers = {
    'Content-Type':'application/json',
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret,   
}
headers

{'Content-Type': 'application/json',
 'X-Naver-Client-Id': 'ZJN9SurWPIxPbNA5Da3w',
 'X-Naver-Client-Secret': 'hU1LDvAjBA'}

In [7]:
response = requests.post(url, params, headers=headers)
response

<Response [400]>

In [8]:
response.text

'{"errorCode":"-10001","errorMessage":"INVALID_REQUEST"}'

In [9]:
import json
response = requests.post(url, json.dumps(params), headers=headers)
response

<Response [200]>

In [10]:
params, json.dumps(params)

({'source': 'ko', 'target': 'en', 'text': '데이터 크롤링은 돈이 됩니다.'},
 '{"source": "ko", "target": "en", "text": "\\ub370\\uc774\\ud130 \\ud06c\\ub864\\ub9c1\\uc740 \\ub3c8\\uc774 \\ub429\\ub2c8\\ub2e4."}')

In [11]:
response.text

'{"message":{"result":{"srcLangType":"ko","tarLangType":"en","translatedText":"Data crawling makes money.","engineType":"N2MT"},"@type":"response","@service":"naverservice.nmt.proxy","@version":"1.0.0"}}'

In [14]:
response.json()['message']

{'result': {'srcLangType': 'ko',
  'tarLangType': 'en',
  'translatedText': 'Data crawling makes money.',
  'engineType': 'N2MT'},
 '@type': 'response',
 '@service': 'naverservice.nmt.proxy',
 '@version': '1.0.0'}

In [15]:
response.json()['message']['result']

{'srcLangType': 'ko',
 'tarLangType': 'en',
 'translatedText': 'Data crawling makes money.',
 'engineType': 'N2MT'}

In [16]:
response.json()['message']['result']['translatedText']

'Data crawling makes money.'

In [17]:
# json > list,dict 
en_txt = response.json()['message']['result']['translatedText']
en_txt

'Data crawling makes money.'

In [18]:
# def
def translate(ko_txt):
    client_id = 'ZJN9SurWPIxPbNA5Da3w'
    client_secret = 'hU1LDvAjBA'
    url = 'https://openapi.naver.com/v1/papago/n2mt'
    params = {
    'source' : 'ko',
    'target' : 'en',
    'text' : ko_txt,
    }
    headers = {
    'Content-Type':'application/json',
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret,   
    }
    response = requests.post(url, json.dumps(params), headers=headers)
    return response.json()['message']['result']['translatedText']

In [19]:
translate('데이터사이언스는 유용합니다')

'Data science is useful'

In [20]:
# 한글 엑셀 파일 영문으로 만들기
%ls

 C 드라이브의 볼륨: WINDOWS
 볼륨 일련 번호: 1CE2-C115

 C:\Users\user\Web_Crawling\day1 디렉터리

2023-08-24  오후 03:23    <DIR>          .
2023-08-24  오전 09:45    <DIR>          ..
2023-08-24  오후 02:31    <DIR>          .ipynb_checkpoints
2023-08-24  오전 11:09            31,038 00_class.ipynb
2023-08-24  오후 02:30           525,443 01_requests_naver_stock.ipynb
2023-08-24  오후 02:52            57,474 02_requests_daum_exchange.ipynb
2023-08-24  오후 03:23             9,682 03_naver_api.ipynb
2023-08-24  오후 02:33             2,554 04_requests_zigbang.ipynb
2023-08-24  오후 02:53            10,060 covid.xlsx
               6개 파일             636,251 바이트
               3개 디렉터리  169,922,547,712 바이트 남음


In [22]:
df = pd.read_excel('covid.xlsx')[['category','title']]
df.tail(2)

,category,title
4,104,"""코로나 확진자 '0명'인 날은 절대 오지 않는다"" 美전문가"
5,104,올림픽 다가오는데 도쿄 코로나 확산 빨라져...일주일새 25％↑


In [23]:
# Series.apply(function) : Series의 모든 데이터를 funtion에 적용한 결과를 출력하는 메서드

In [25]:
en_title = df['title'].apply(translate)

In [26]:
en_title

0    SK Bioscience Submits Phase 3 Trial Plan for C...
1    41 cumulative confirmed cases of COVID-19 at t...
2    New COVID-19 infections, 542 people by 9 p.m. ...
3    A confirmed case of COVID-19 occurred at a clu...
4    "The day when there are '0' confirmed cases of...
5    As the Olympics are approaching, the spread of...
Name: title, dtype: object

In [27]:
df['en_title'] = en_title
df.tail(2)

,category,title,en_title
4,104,"""코로나 확진자 '0명'인 날은 절대 오지 않는다"" 美전문가","""The day when there are '0' confirmed cases of..."
5,104,올림픽 다가오는데 도쿄 코로나 확산 빨라져...일주일새 25％↑,"As the Olympics are approaching, the spread of..."


In [29]:
df.to_excel('en_covid.xlsx',index=False, encoding='utf-8-sig')

In [30]:
%ls

 C 드라이브의 볼륨: WINDOWS
 볼륨 일련 번호: 1CE2-C115

 C:\Users\user\Web_Crawling\day1 디렉터리

2023-08-24  오후 03:43    <DIR>          .
2023-08-24  오전 09:45    <DIR>          ..
2023-08-24  오후 02:31    <DIR>          .ipynb_checkpoints
2023-08-24  오전 11:09            31,038 00_class.ipynb
2023-08-24  오후 02:30           525,443 01_requests_naver_stock.ipynb
2023-08-24  오후 02:52            57,474 02_requests_daum_exchange.ipynb
2023-08-24  오후 03:41            14,566 03_naver_api.ipynb
2023-08-24  오후 02:33             2,554 04_requests_zigbang.ipynb
2023-08-24  오후 02:53            10,060 covid.xlsx
2023-08-24  오후 03:43             5,708 en_covid.xlsx
               7개 파일             646,843 바이트
               3개 디렉터리  169,919,934,464 바이트 남음


#### 2. 통합검색어 트렌드 api
- 서비스 : `https://datalab.naver.com/keyword/trendSearch.naver`
- 내 애플리케이션 > dss 애플리케이션 > API 설정 > 데이터랩(검색어 트렌드) 추가
- 사용법 : `https://developers.naver.com/docs/serviceapi/datalab/search/search.md#통합-검색어-트렌드`